In [1]:
import os, code, sys, time, argparse

import numpy as np
from sklearn.neighbors import kneighbors_graph
import tensorflow as tf

import utils
import nn
from utils import REDSHIFTS, PARAMS_SEED, LEARNING_RATE, RS_TAGS

In [2]:
n_P = 32
N = n_P**3
M = 14
k = 3
b = 4
redshifts = [15, 19]

In [3]:
# Load data
X = utils.load_zuni_npy_data(redshifts=redshifts, norm_coo=True)[...,:-1]
Y = X[1] # (1000, N, 6)
X = X[0]

LD: X_0.2280_.npy
LD: X_0.0000_.npy


In [4]:
_x_batch = utils.next_minibatch(X[None,...], b, data_aug=False) # shape (2, b, N, 6)

# split data
x_in    = _x_batch[0] # (b, N, 6)
#x_truth = _x_batch[1] # (b, N, 6)

def graph_get_func(h_in, offset=False, inc_self=False): # for tf.py_func
    return nn.get_kneighbor_list(h_in[...,:3], M, offset_idx=offset, inc_self=inc_self)

# get adj_list
#adj_list = graph_get_func(x_in) # (b, N, M)
#ilist    = nn.alist_to_indexlist(adj_list)
adj_csrs = graph_get_func(x_in)
adj_csrs_offset = graph_get_func(x_in, offset=True)

# get edges (relative dists) and nodes (velocities)
#x_in_edges = nn.get_input_edge_features(x_in, adj_list) # (b, N, M, 3)
#x_in_nodes = nn.get_input_node_features(x_in) # (b, N, 3)

In [7]:
edges = nn.get_input_edge_features_batch(x_in, adj_csrs, M)
edges_off = nn.get_input_edge_features_batch_offset(x_in, adj_csrs_offset, M)
np.all(edges == edges_off)

True

In [8]:
sess = tf.InteractiveSession()

In [9]:
def usm(x, idx, num_seg):
    return tf.unsorted_segment_mean(x, idx, num_seg).eval()

In [46]:
np.random.permutation(np.arange(24))

array([ 7, 22,  2,  3, 13, 14, 16,  0, 18,  1,  9,  8, 17, 20, 12,  5, 10,
       15,  6, 19, 11, 23,  4, 21])

In [73]:
c = 3
P = 6
Q = 4
h = np.random.randint(0,30,(c, P*Q, 3)) # N: 6, M:4
#idx = np.random.randint(0,4,(4,24)) # (b, N*M)
idx = np.zeros((c, P, Q)).astype(np.int32)
offset_idx = np.zeros((c, P, Q)).astype(np.int32)
valid_idx = np.arange(P)
for i in range(c):
    for j in range(P):
        cur_idx = np.random.choice(valid_idx, (Q,), replace=False)
        idx[i, j] = cur_idx
        offset_idx[i,j] = cur_idx + P*i
idx.shape

(3, 6, 4)

In [74]:
idx_in = idx.reshape(c, P*Q)

In [75]:
per_batch = P * np.expand_dims(np.arange(c), axis=1) # (c,1): [[0], [6], [12]]
idx_per_batch = idx_in + per_batch #
idx_per_batch.shape

(3, 24)

In [77]:
usm(h, idx_in, c*P)

array([[13.28571429, 12.14285714, 16.21428571],
       [12.63636364, 15.54545455, 17.72727273],
       [16.81818182, 15.90909091, 16.63636364],
       [16.81818182,  8.        , 13.36363636],
       [16.46666667, 13.        , 15.26666667],
       [17.6       , 11.9       , 11.5       ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [76]:
usm(h, idx_per_batch, c*P)

array([[19.25      , 15.        , 24.25      ],
       [12.66666667, 19.66666667, 18.        ],
       [19.4       , 13.6       , 24.8       ],
       [17.66666667,  5.33333333, 13.33333333],
       [19.25      , 16.25      , 22.5       ],
       [14.        , 14.5       , 20.        ],
       [ 8.25      ,  9.        , 14.        ],
       [ 3.5       , 20.5       , 27.5       ],
       [14.75      , 18.5       , 11.75      ],
       [13.25      ,  7.        , 20.75      ],
       [15.33333333,  9.66666667, 11.83333333],
       [19.25      ,  7.75      , 13.25      ],
       [12.66666667, 12.33333333, 12.33333333],
       [18.66666667,  4.        , 10.66666667],
       [14.5       , 16.5       ,  6.        ],
       [19.75      , 11.        ,  6.        ],
       [15.6       , 14.4       , 13.6       ],
       [17.75      , 14.75      ,  5.5       ]])

In [79]:
offset_idx_r = np.reshape(offset_idx, (c, P*Q))

In [81]:
usm(h, offset_idx_r, c*P)

array([[19.25      , 15.        , 24.25      ],
       [12.66666667, 19.66666667, 18.        ],
       [19.4       , 13.6       , 24.8       ],
       [17.66666667,  5.33333333, 13.33333333],
       [19.25      , 16.25      , 22.5       ],
       [14.        , 14.5       , 20.        ],
       [ 8.25      ,  9.        , 14.        ],
       [ 3.5       , 20.5       , 27.5       ],
       [14.75      , 18.5       , 11.75      ],
       [13.25      ,  7.        , 20.75      ],
       [15.33333333,  9.66666667, 11.83333333],
       [19.25      ,  7.75      , 13.25      ],
       [12.66666667, 12.33333333, 12.33333333],
       [18.66666667,  4.        , 10.66666667],
       [14.5       , 16.5       ,  6.        ],
       [19.75      , 11.        ,  6.        ],
       [15.6       , 14.4       , 13.6       ],
       [17.75      , 14.75      ,  5.5       ]])

In [72]:
offset_idx.shape

(4,)